# Yelp Data Challenge - Restaurant Recommender

Yujun Zhou

Sep 2018

* Prepare utility matrix

* Use item-item collaborative filtering to build recommender system

* Use matrix factorization to build recommender system


In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
plt.style.use("ggplot")

In [11]:
df = pd.read_csv('clean_data/last_2_years_restaurant_reviews.csv')

In [12]:
df.head()

,business_id,name,categories,avg_stars,cool,date,funny,review_id,stars,text,useful,user_id
0,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"Steakhouses, Restaurants, Cajun/Creole",4.0,0,2017-02-14,0,VETXTwMw6qxzOVDlXfe6Tg,5,went for dinner tonight. Amazing my husband ha...,0,ymlnR8UeFvB4FZL56tCZsA
1,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"Steakhouses, Restaurants, Cajun/Creole",4.0,0,2017-12-04,0,S8-8uZ7fa5YbjnEtaW15ng,5,This was an amazing dinning experience! ORDER ...,0,9pSSL6X6lFpY3FCRLEH3og
2,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"Steakhouses, Restaurants, Cajun/Creole",4.0,0,2016-08-22,1,1nK5w0VNfDlnR3bOz13dJQ,5,My husband and I went there for lunch on a Sat...,1,gm8nNoA3uB4In5o_Hxpq3g
3,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"Steakhouses, Restaurants, Cajun/Creole",4.0,0,2016-09-13,0,N1Z93BthdJ7FT2p5S22jIA,3,Went for a nice anniversary dinner. Researched...,0,CEtidlXNyQzgJSdF1ubPFw
4,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"Steakhouses, Restaurants, Cajun/Creole",4.0,0,2015-02-02,0,_Uwp6FO1X-avE9wqTMC59w,5,This place is first class in every way. Lobste...,0,-Z7Nw2UF7NiBSAzfXNA_XA


## 1. Clean data and get rating data 

#### Select relevant columns in the original dataframe

In [15]:
# Get business_id, user_id, stars for recommender
select = ['business_id','user_id','stars']
df_relevant = df[select]
df_relevant

,business_id,user_id,stars
0,--9e1ONYQuAa-CB_Rrw7Tw,ymlnR8UeFvB4FZL56tCZsA,5
1,--9e1ONYQuAa-CB_Rrw7Tw,9pSSL6X6lFpY3FCRLEH3og,5
2,--9e1ONYQuAa-CB_Rrw7Tw,gm8nNoA3uB4In5o_Hxpq3g,5
3,--9e1ONYQuAa-CB_Rrw7Tw,CEtidlXNyQzgJSdF1ubPFw,3
4,--9e1ONYQuAa-CB_Rrw7Tw,-Z7Nw2UF7NiBSAzfXNA_XA,5
5,--9e1ONYQuAa-CB_Rrw7Tw,knB3eczyruOuZaP3TKUKew,5
6,--9e1ONYQuAa-CB_Rrw7Tw,9_BhDyzJYf2JwTD9TyXJ4g,5
7,--9e1ONYQuAa-CB_Rrw7Tw,cZVQGCZ_fHtTdfiyGVJPdg,5
8,--9e1ONYQuAa-CB_Rrw7Tw,4DrtQoV20cbDOlnOk_rxHw,3
9,--9e1ONYQuAa-CB_Rrw7Tw,rhP3yu4Sxpj9OXgeY0VpLg,5


#### There are many users that haven't given many reviews, exclude these users from the item-item similarity recommender

**Q**: How do we recommend to these users anyways?

In [ ]:
# To be implemented


#### Create utility matrix from records

In [ ]:
# To be implemented

highest_user_id = df_ratings_contents.user.max()
highest_movie_id = df_ratings_contents.movie.max()
ratings_mat = sparse.lil_matrix((highest_user_id, highest_movie_id))
ratings_mat

## 2. Item-Item similarity recommender

### reuse the ItemItemRecommender class derived from previous exercise

 make modification to accommodate the dense numpy array

In [ ]:
# To be implemented

pass

## 3. Matrix Factorization recommender

##### Compare two of the methods demoed in Practice Class: sklearn NMF, sklearn TruncatedSVD, or GraphLab
##### *Extra points for using GraphLab

## 4. Other recommenders 

 other ways to build a better recommender?

* Other features (have you noticed there are other features in the Yelp dataset, e.g. tips, etc.?)
* Popularity-based
* Content-based
* Hybrid